In [ ]:
#"Does investing in education reduce STI risk in California?"
#Conclusion

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [94]:
df = pd.read_csv('stds-by-disease-county-year-sex.csv',  encoding='windows-1252')
df.columns

Index(['Disease', 'County', 'Year', 'Sex', 'Cases', 'Population', 'Rate',
       'Lower 95% CI', 'Upper 95% CI', 'Annotation Code'],
      dtype='object')

In [95]:
#drop columns
df = df.drop(columns=['Annotation Code', 'Lower 95% CI', 'Upper 95% CI'])
print(df.columns)
print(df.shape)
df.head(9)

Index(['Disease', 'County', 'Year', 'Sex', 'Cases', 'Population', 'Rate'], dtype='object')
(10620, 7)


,Disease,County,Year,Sex,Cases,Population,Rate
0,Chlamydia,California,2001,Female,75941.0,17339700,438.0
1,Chlamydia,California,2001,Male,24885.0,17173042,144.9
2,Chlamydia,California,2001,Total,101590.0,34512742,294.4
3,Chlamydia,California,2002,Female,81583.0,17554666,464.7
4,Chlamydia,California,2002,Male,28521.0,17383624,164.1
5,Chlamydia,California,2002,Total,110759.0,34938290,317.0
6,Chlamydia,California,2003,Female,85153.0,17782868,478.8
7,Chlamydia,California,2003,Male,31007.0,17606060,176.1
8,Chlamydia,California,2003,Total,116385.0,35388928,328.9


In [96]:
df2 = pd.read_csv('County_-_Expenditures.csv',  encoding='windows-1252')
df2.columns

Index(['Entity Name', 'Fiscal Year', 'Type', 'Form/Table', 'Category',
       'Subcategory 1', 'Subcategory 2', 'Line Description', 'Values',
       'Zip Code', 'Estimated Population', 'Area in Sq. Miles', 'Row Number'],
      dtype='object')

In [97]:
#select columns
df2 = df2 [['Fiscal Year', 'Category', 'Entity Name', 'Values']]
df2=df2.rename({'Fiscal Year':'Year', 'Category':'Expense','Entity Name':'County'}, axis=1)
#Only expense chosen in Education and the values need to be > 0
df2 = df2[(df2['Expense'].str.contains('Education')) & (df2['Values'] > 0)]
#rename rows
df2['Expense'] = df2['Expense'].replace({'Education and Recreation and Cultural Services': 'Education'})
df2

,Year,Expense,County,Values
551,2019,Education,Alameda,30071131
555,2019,Education,Alameda,338671
575,2019,Education,Alameda,800934
1194,2019,Education,Alpine,65073
1199,2019,Education,Alpine,316422
...,...,...,...,...
341725,2021,Education,Yolo,1230013
341730,2021,Education,Yolo,132059
342356,2021,Education,Yuba,648615
342361,2021,Education,Yuba,100374


In [98]:
df2 = df2.groupby(['Year', 'Expense', 'County']).agg('sum').reset_index()
df2

,Year,Expense,County,Values
0,2003,Education,Alameda,19774911
1,2003,Education,Alpine,409598
2,2003,Education,Amador,823187
3,2003,Education,Butte,2513026
4,2003,Education,Calaveras,694265
...,...,...,...,...
1076,2021,Education,Tulare,8551779
1077,2021,Education,Tuolumne,1410279
1078,2021,Education,Ventura,9519420
1079,2021,Education,Yolo,10039838


In [99]:
df3 = pd.merge(df2, df, how='inner', on =['County','Year'])
df3 = df3.drop(columns=['Expense'])
print("df3",df3.shape)
print("df",df.shape)
df3.head(10)


df3 (9225, 8)
df (10620, 7)


,Year,County,Values,Disease,Sex,Cases,Population,Rate
0,2003,Alameda,19774911,Chlamydia,Female,3780.0,747441,505.7
1,2003,Alameda,19774911,Chlamydia,Male,1143.0,719746,158.8
2,2003,Alameda,19774911,Chlamydia,Total,4928.0,1467187,335.9
3,2003,Alameda,19774911,Early Syphilis,Female,3.0,747441,0.4
4,2003,Alameda,19774911,Early Syphilis,Male,61.0,719746,8.5
5,2003,Alameda,19774911,Early Syphilis,Total,64.0,1467187,4.4
6,2003,Alameda,19774911,Gonorrhea,Female,842.0,747441,112.7
7,2003,Alameda,19774911,Gonorrhea,Male,791.0,719746,109.9
8,2003,Alameda,19774911,Gonorrhea,Total,1635.0,1467187,111.4
9,2003,Alpine,409598,Chlamydia,Female,NaN,596,NaN


In [100]:
#Shows only total in sex
df_totals=df3[df['Sex'].str.contains('Total')].reset_index()
df_totals

C:\Users\danie\AppData\Local\Temp\ipykernel_23068\2015068020.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,index,Year,County,Values,Disease,Sex,Cases,Population,Rate
0,2,2003,Alameda,19774911,Chlamydia,Total,4928.0,1467187,335.9
1,5,2003,Alameda,19774911,Early Syphilis,Total,64.0,1467187,4.4
2,8,2003,Alameda,19774911,Gonorrhea,Total,1635.0,1467187,111.4
3,11,2003,Alpine,409598,Chlamydia,Total,3.0,1249,240.2
4,14,2003,Alpine,409598,Early Syphilis,Total,0.0,1249,0.0
...,...,...,...,...,...,...,...,...,...
3070,9212,2020,Yolo,9565783,Early Syphilis,Total,60.0,216544,27.7
3071,9215,2020,Yolo,9565783,Gonorrhea,Total,302.0,216544,139.5
3072,9218,2020,Yuba,1195070,Chlamydia,Total,293.0,81468,359.7
3073,9221,2020,Yuba,1195070,Early Syphilis,Total,75.0,81468,92.1


In [101]:
#removes county
df_totals_california=df_totals.groupby(['Year','Disease']).agg('sum').reset_index()
df_totals_california['Rate'] = (df_totals_california['Cases'] /df_totals_california ['Population'])*100000
df_totals_california

C:\Users\danie\AppData\Local\Temp\ipykernel_23068\1861523816.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Year,Disease,index,Values,Cases,Population,Rate
0,2003,Chlamydia,14478,361444891,113053.0,34607058,326.676136
1,2003,Early Syphilis,14649,361444891,1598.0,34607058,4.617555
2,2003,Gonorrhea,14820,361444891,23883.0,34607058,69.011934
3,2004,Chlamydia,43719,374040953,119821.0,34972066,342.619164
4,2004,Early Syphilis,43890,374040953,1709.0,34972066,4.886757
5,2004,Gonorrhea,44061,374040953,28339.0,34972066,81.033245
6,2005,Chlamydia,72960,388158152,125380.0,35205927,356.133216
7,2005,Early Syphilis,73131,388158152,2359.0,35205927,6.700576
8,2005,Gonorrhea,73302,388158152,31636.0,35205927,89.859869
9,2006,Chlamydia,102201,420068555,132388.0,35463894,373.303620


In [102]:
# Chlamydia - correlation between Rate and Values invested in education
dfc = df_totals[df_totals['Disease'].str.contains('Chlamydia',case=False) ].reset_index()
# Assuming you have a DataFrame 'df3' with 'County', 'Values', and 'Rate' columns

# Calculate correlations
correlation_dfc = dfc.groupby('County')['Values', 'Rate'].corr().reset_index()
correlation_dfc = correlation_dfc[correlation_dfc['level_1'] == 'Values'][['County', 'Rate']]

# Rename columns
correlation_dfc.columns = ['County', 'Correlation']

# Print the resulting table
correlation_dfc

C:\Users\danie\AppData\Local\Temp\ipykernel_23068\3895871780.py:6: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,County,Correlation
0,Alameda,0.642209
2,Alpine,0.432019
4,Amador,0.090316
6,Butte,0.543655
8,Calaveras,-0.255723
10,Colusa,0.603937
12,Contra Costa,0.676961
14,Del Norte,0.641644
16,El Dorado,0.611384
18,Fresno,0.377339


In [103]:
# Early Syphilis -correlation between Rate and Values invested in education
dfs = df_totals[df_totals['Disease'].str.contains('Early Syphilis')].reset_index()
# Calculate correlations
correlation_dfs = dfs.groupby('County')['Values', 'Rate'].corr().reset_index()
correlation_dfs = correlation_dfs[correlation_dfs['level_1'] == 'Values'][['County', 'Rate']]

# # Rename columns
correlation_dfs.columns = ['County', 'Correlation']

# # Print the resulting table
correlation_dfs


C:\Users\danie\AppData\Local\Temp\ipykernel_23068\792075654.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,County,Correlation
0,Alameda,0.926220
2,Alpine,NaN
4,Amador,0.236633
6,Butte,0.827960
8,Calaveras,-0.132484
10,Colusa,0.629635
12,Contra Costa,0.812475
14,Del Norte,0.544076
16,El Dorado,0.834689
18,Fresno,0.260419


In [104]:
#Gonorrhea -correlation between Rate and Values invested in education
dfg = df_totals[df_totals['Disease'].str.contains('Gonorrhea')].reset_index()
# Calculate correlations
correlation_dfg = dfg.groupby('County')['Values', 'Rate'].corr().reset_index()
correlation_dfg = correlation_dfg[correlation_dfg['level_1'] == 'Values'][['County', 'Rate']]

# # Rename columns
correlation_dfg.columns = ['County', 'Correlation']

# # Print the resulting table
correlation_dfg

C:\Users\danie\AppData\Local\Temp\ipykernel_23068\1560895655.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,County,Correlation
0,Alameda,0.686015
2,Alpine,0.186398
4,Amador,0.545021
6,Butte,0.679105
8,Calaveras,0.363796
10,Colusa,0.669674
12,Contra Costa,0.806235
14,Del Norte,0.740346
16,El Dorado,0.713317
18,Fresno,0.059712


In [105]:
#Merging the 3 df os correlations together

df_correlations = pd.merge(correlation_dfc, correlation_dfs, how='inner', on =['County'])
df_correlations = pd.merge(df_correlations, correlation_dfg, how='inner', on =['County'])
df_correlations = df_correlations.rename({'Correlation_x':'Chlamydia', 'Correlation_y':'Early Syphilis','Correlation':'Gonorrhea'}
                                         , axis=1)
df_correlations 

,County,Chlamydia,Early Syphilis,Gonorrhea
0,Alameda,0.642209,0.926220,0.686015
1,Alpine,0.432019,NaN,0.186398
2,Amador,0.090316,0.236633,0.545021
3,Butte,0.543655,0.827960,0.679105
4,Calaveras,-0.255723,-0.132484,0.363796
5,Colusa,0.603937,0.629635,0.669674
6,Contra Costa,0.676961,0.812475,0.806235
7,Del Norte,0.641644,0.544076,0.740346
8,El Dorado,0.611384,0.834689,0.713317
9,Fresno,0.377339,0.260419,0.059712


In [106]:
import plotly.graph_objects as go

fig = px.box(df_correlations,  y=["Chlamydia",'Early Syphilis', 'Gonorrhea'],  points='all',hover_name='County')

fig.show()

In [107]:
# Sort the DataFrame and calculate growth rate
df_totals_california_s = df_totals_california.sort_values(['Disease', 'Year'])
df_totals_california_s['Growth_Rate'] = df_totals_california_s['Rate'].pct_change(periods=1) * 100
df_totals_california_s.loc[df_totals_california_s['Year'] == 2003, 'Growth_Rate'] = None  # Set growth rate to None for year 2003

# Display the modified DataFrame
df_totals_california_s

,Year,Disease,index,Values,Cases,Population,Rate,Growth_Rate
0,2003,Chlamydia,14478,361444891,113053.0,34607058,326.676136,NaN
3,2004,Chlamydia,43719,374040953,119821.0,34972066,342.619164,4.880377
6,2005,Chlamydia,72960,388158152,125380.0,35205927,356.133216,3.944336
9,2006,Chlamydia,102201,420068555,132388.0,35463894,373.303620,4.821343
12,2007,Chlamydia,131442,465557639,139075.0,35761195,388.899196,4.177719
15,2008,Chlamydia,160683,499335099,145153.0,36057549,402.559253,3.512493
18,2009,Chlamydia,189924,483138341,143430.0,36275405,395.391864,-1.780456
21,2010,Chlamydia,219165,448348040,150738.0,36553423,412.377249,4.295836
24,2011,Chlamydia,248406,439171885,159811.0,36884415,433.275138,5.067663
27,2012,Chlamydia,277647,415642826,164573.0,37224317,442.111537,2.039443


In [109]:
import plotly.express as px

values_df = df_totals_california_s[:18]
values_df

fig = px.line(values_df, x="Year", y='Values',markers=True, 
              title="Value invested in education")

fig.show()

In [110]:
import plotly.express as px

fig = px.line(df_totals_california_s, x="Year", y='Cases', color='Disease',markers=True, 
              title="Cases in each Year for each Disease")

fig.show()

In [119]:
import plotly.express as px

fig = px.line(df_totals_california_s, x="Year", y='Growth_Rate', color='Disease',markers=True, 
              title="Growth Rate in each Year for each Disease")

fig.show()

In [ ]:
df_sex=df3.groupby(['Year','Disease','Sex']).agg('sum').reset_index()
df_sex['Rate'] = (df_sex['Cases'] /df_sex ['Population'])*100000
df_sex

In [120]:
import plotly.express as px
gonorrhea_df = df_sex[df_sex['Disease'] == 'Gonorrhea']

fig = px.bar(gonorrhea_df, x='Year', y='Rate', color='Sex',
             barmode='group', facet_col='Disease',
             title='Gonorrhea Rates by Year, Disease, and Sex')

chlamydia_df = df_sex[df_sex['Disease'] == 'Chlamydia']

fig2 = px.bar(chlamydia_df, x='Year', y='Rate', color='Sex',
             barmode='group', facet_col='Disease',
             title='Gonorrhea Rates by Year, Disease, and Sex')

early_syphilis_df = df_sex[df_sex['Disease'] == 'Early Syphilis']

fig3 = px.bar(early_syphilis_df, x='Year', y='Rate', color='Sex',
             barmode='group', facet_col='Disease',
             title='Gonorrhea Rates by Year, Disease, and Sex')

fig.show()
fig2.show()
fig3.show()

In [ ]:
correlation = df_totals_california[df_totals_california['Disease'] == 'Chlamydia']['Cases'].corr(df_totals_california[df_totals_california['Disease'] == 'Chlamydia']['Values'])
correlation2 = df_totals_california[df_totals_california['Disease'] == 'Early Syphilis']['Cases'].corr(df_totals_california[df_totals_california['Disease'] == 'Early Syphilis']['Values'])
correlation3 = df_totals_california[df_totals_california['Disease'] == 'Gonorrhea']['Cases'].corr(df_totals_california[df_totals_california['Disease'] == 'Gonorrhea']['Values'])

print('Chlamydia',correlation)
print('Early Syphilis',correlation2)
print('Gonorrhea',correlation3)

In [ ]:
np.sum(df_totals_california.isnull())
#no missing data

In [ ]:
df3.groupby(['Disease','Year']).agg(['sum'])['Cases']

In [ ]:
import plotly.express as px
fig = px.scatter(df_totals, x="Rate", y="Values", animation_frame='Year', animation_group="County",
           color="Disease", hover_name="County", facet_col="Disease",
           log_x=False)
fig.show()

In [ ]:
import plotly.graph_objects as go

df_p = df3[df3['Year'].astype(str).str.contains('2019')].reset_index()
 
fig = px.box(df_p, x="Disease", y="Rate", color='Sex', points='all',hover_name='County')

fig.show()